In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
import tkinter as tk
from tkinter import ttk, messagebox
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
import seaborn as sns
from sklearn.metrics import confusion_matrix, accuracy_score

df = pd.read_csv(r"Internet Speed.csv")
max_speed = df['Internet_speed'].max()
bins = [0, 500, 1500, 2500, max_speed + 1]
labels = ["Very Unstable", "Unstable", "Stable", "Very Stable"]
df['Stability_Category'] = pd.cut(df['Internet_speed'], bins=bins, labels=labels)
df['Stability_Category'].replace(["Very Unstable", "Unstable", "Stable", "Very Stable"], [1, 2, 3, 4], inplace=True)

essential_features = ["Ping_latency", "Download_speed", "Upload_speed", "Packet_loss_rate", "Signal_strength"]
x = df[essential_features]
y = df["Stability_Category"]

x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.8, random_state=42)

models = {
    "Logistic Regression": LogisticRegression(max_iter=1000),
    "Gaussian NB": GaussianNB(),
    "Multinomial NB": MultinomialNB(),
    "Decision Tree": DecisionTreeClassifier(),
    "Linear SVM": SVC(kernel='rbf'),
    "Non-Linear SVM": SVC(kernel='poly')
}
model_accuracies = {}
for name, model in models.items():
    model.fit(x_train, y_train)
    y_pred = model.predict(x_test)
    model_accuracies[name] = accuracy_score(y_test, y_pred) * 100

# Tkinter UI
class InternetStabilityPredictor:
    def __init__(self, root):
        self.root = root
        self.root.title("Internet Stability Predictor")
        self.root.geometry("700x750")
        self.root.configure(bg="#f0f0f0")

        # Welcome Frame
        welcome_frame = tk.Frame(root, bg="#e0e0e0", pady=10)
        welcome_frame.pack(fill="x")
        tk.Label(welcome_frame, text="Welcome to Internet Stability Predictor!", font=("Arial", 16, "bold"), bg="#e0e0e0").pack()
        tk.Label(welcome_frame, text="Enter your internet details to predict stability.", font=("Arial", 10), bg="#e0e0e0").pack()

        # Input Frame
        input_frame = tk.Frame(root, bg="#f0f0f0", pady=20)
        input_frame.pack()
        self.entries = {}
        feature_descriptions = {
            "Ping_latency": "Ping (ms): Time for data to travel (e.g., 20)",
            "Download_speed": "Download Speed (Mbps): Speed of downloading (e.g., 100)",
            "Upload_speed": "Upload Speed (Mbps): Speed of uploading (e.g., 50)",
            "Packet_loss_rate": "Packet Loss (%): Data loss rate (e.g., 0.5)",
            "Signal_strength": "Signal Strength (%): Connection quality (e.g., 80)"
        }
        for i, (feature, desc) in enumerate(feature_descriptions.items()):
            tk.Label(input_frame, text=desc, bg="#f0f0f0").grid(row=i, column=0, padx=10, pady=5, sticky="w")
            entry = tk.Entry(input_frame)
            entry.grid(row=i, column=1, padx=10, pady=5)
            self.entries[feature] = entry

        # Model Selection Frame
        model_frame = tk.Frame(root, bg="#f0f0f0", pady=10)
        model_frame.pack()
        tk.Label(model_frame, text="Choose a Prediction Model:", bg="#f0f0f0").grid(row=0, column=0, padx=10)
        self.model_var = tk.StringVar()
        model_dropdown = ttk.Combobox(model_frame, textvariable=self.model_var, values=list(models.keys()))
        model_dropdown.grid(row=0, column=1, padx=10)
        model_dropdown.set("Logistic Regression")

        # Buttons Frame
        button_frame = tk.Frame(root, bg="#f0f0f0", pady=20)
        button_frame.pack()
        tk.Button(button_frame, text="Predict Stability", command=self.predict, bg="green", fg="white", font=("Arial", 10, "bold")).grid(row=0, column=0, padx=20)
        tk.Button(button_frame, text="Show Prediction Distribution", command=self.show_distribution).grid(row=0, column=1, padx=20)
        tk.Button(button_frame, text="Show Confusion Matrix", command=self.show_confusion_matrix).grid(row=0, column=2, padx=20)

        # Result Frame
        self.result_frame = tk.Frame(root, bg="#f0f0f0", pady=20)
        self.result_frame.pack()
        self.result_label = tk.Label(self.result_frame, text="Prediction will appear here", font=("Arial", 12), bg="#f0f0f0")
        self.result_label.pack()
        self.accuracy_label = tk.Label(self.result_frame, text="", font=("Arial", 10), bg="#f0f0f0")
        self.accuracy_label.pack()

        # About Frame
        about_frame = tk.Frame(root, bg="#d0d0d0", pady=10)
        about_frame.pack(fill="x", side="bottom")
        tk.Label(about_frame, text="About: This tool predicts internet stability (Very Unstable to Very Stable) using machine learning models based on 5 key inputs.", 
                 font=("Arial", 9), bg="#d0d0d0", wraplength=650).pack()

    def predict(self):
        try:
            inputs = []
            for feature in self.entries:
                value = self.entries[feature].get()
                if value == "":
                    raise ValueError(f"Please enter a value for {feature}")
                inputs.append(float(value))

            input_array = np.array(inputs).reshape(1, -1)
            selected_model = models[self.model_var.get()]
            prediction = selected_model.predict(input_array)[0]
            stability = {1: "Very Unstable", 2: "Unstable", 3: "Stable", 4: "Very Stable"}
            result = stability[prediction]

            self.result_label.config(text=f"Predicted Stability: {result}", fg="blue")
            self.accuracy_label.config(text=f"Model Accuracy: {model_accuracies[self.model_var.get()]:.2f}%", fg="green")
        except ValueError as e:
            messagebox.showerror("Input Error", str(e))
        except Exception as e:
            messagebox.showerror("Error", "An error occurred during prediction.")

    def show_distribution(self):
        selected_model = models[self.model_var.get()]
        y_pred = selected_model.predict(x_test)
        classes, counts = np.unique(y_pred, return_counts=True)
        
        fig = plt.Figure(figsize=(5, 4))
        ax = fig.add_subplot(111)
        sns.barplot(x=classes, y=counts, palette=['blue', 'green', 'orange', 'red'], ax=ax)
        ax.set_xlabel("Stability Class (1=Very Unstable, 4=Very Stable)")
        ax.set_ylabel("Count")
        ax.set_title(f"Prediction Distribution - {self.model_var.get()}")
        self.show_plot(fig, "Prediction Distribution")

    def show_confusion_matrix(self):
        selected_model = models[self.model_var.get()]
        y_pred = selected_model.predict(x_test)
        cm = confusion_matrix(y_test, y_pred)
        
        fig = plt.Figure(figsize=(5, 4))
        ax = fig.add_subplot(111)
        sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", ax=ax,
                    xticklabels=labels, yticklabels=labels)
        ax.set_xlabel("Predicted")
        ax.set_ylabel("Actual")
        ax.set_title(f"Confusion Matrix - {self.model_var.get()}")
        self.show_plot(fig, "Confusion Matrix")

    def show_plot(self, fig, title):
        new_window = tk.Toplevel(self.root)
        new_window.title(title)
        canvas = FigureCanvasTkAgg(fig, master=new_window)
        canvas.draw()
        canvas.get_tk_widget().pack()
        new_window.geometry("500x400")

# Run the application
if __name__ == "__main__":
    root = tk.Tk()
    app = InternetStabilityPredictor(root)
    root.mainloop()

C:\Users\prath\AppData\Local\Temp\ipykernel_35020\3614306307.py:20: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Stability_Category'].replace(["Very Unstable", "Unstable", "Stable", "Very Stable"], [1, 2, 3, 4], inplace=True)
C:\Users\prath\AppData\Local\Temp\ipykernel_35020\3614306307.py:20: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_opt